In [ ]:
from qiskit import QuantumCircuit, QuantumRegister, AncillaRegister
from qiskit.circuit.library import MCPhaseGate, QFTGate 
from math import log2, ceil, pi

In [ ]:
def draper_adder(a, num_of_qubits, num_of_controls, exclude_qft=False):
    ## First num_of_controls qubits are control qubits
    circuit = QuantumCircuit(num_of_qubits + num_of_controls, name=f"Add {a} ({num_of_controls}c+{num_of_qubits}q)")

    qft = None
    if not exclude_qft:
        qft = QFTGate(num_of_qubits)
        qft_indices = range(num_of_controls, num_of_qubits+num_of_controls)
        circuit.append(qft, qft_indices)

    for i in range(num_of_qubits):
        current_phase = 2 * pi * a / (2**(num_of_qubits - i))
        if num_of_controls > 0:
            circuit.mcp(current_phase, list(range(0, num_of_controls)), i+num_of_controls)
        else:
            circuit.p(current_phase, i)

    if not exclude_qft:
        circuit.append(qft.inverse(), qft_indices)

    return circuit.to_gate()


In [ ]:
def modular_adder(a, N, num_of_qubits):
    num_of_controls = 2
    circuit = QuantumCircuit(num_of_qubits 
                             + 1 #ancilla qubit
                             + 1 #overflow qubit
                             + num_of_controls, name=f"Modular add {a} mod {N} ({num_of_qubits}q)")

    a_adder = draper_adder(a, num_of_qubits + 1, num_of_controls, exclude_qft=True)
    circuit.append(a_adder, range(0, num_of_controls + num_of_qubits + 1))
    minus_N_adder = draper_adder(-N, num_of_qubits + 1, 0, exclude_qft=True)
    circuit.append(minus_N_adder, range(num_of_controls, num_of_controls + num_of_qubits + 1))
    
    # Perform inverse Quantum Fourier Transform in order to read overflow bit
    qft = QFTGate(num_of_qubits + 1)
    qft_indices = range(num_of_controls, num_of_qubits+num_of_controls+1)
    circuit.append(qft.inverse(), qft_indices)

    overflow_bit_index = num_of_controls + num_of_qubits
    ancilla_bit_index = overflow_bit_index + 1
    # Use the overflow bit to check if we need to add N-a (the inverse of adding a-N)
    circuit.cx(overflow_bit_index, ancilla_bit_index)

    circuit.append(qft, qft_indices)
    # add N back if we weren't greater than N
    N_adder_with_control = draper_adder(N, num_of_qubits + 1, num_of_controls=1, exclude_qft=True)

    # restore ancilla - this is no Joke
    circuit.append(N_adder_with_control, [ancilla_bit_index] + list(qft_indices))
    circuit.append(a_adder.inverse(), range(0, num_of_controls + num_of_qubits + 1))
    circuit.append(qft.inverse(), qft_indices)
    circuit.x(overflow_bit_index)
    circuit.cx(overflow_bit_index, ancilla_bit_index)
    circuit.x(overflow_bit_index)
    circuit.append(qft, qft_indices)
    circuit.append(a_adder, range(0, num_of_controls + num_of_qubits + 1))

    return circuit.to_gate()

In [ ]:
myCircuit = QuantumCircuit(10)

myModAdder = modular_adder(3, 5, 3)

myCircuit.append(myModAdder, list(range(0, 7)))

myCircuit.draw()